In [1]:
# So, we want to be able to classify articles into tech and non-tech using K-NN.
# How will we go about this? (I think I might go for Daily Mail / Guardian instead)

# Additional: If anyone else is reading this, apologies - it could be neater. I wasn't
# expecting to have quite so much fun and go so far with it, hence why it's now 2:20am.

In [2]:
# We'll attempt the following steps:
# 1. Download a relevant corpus - pick a new website and extract two sets of article: tech and sport.
# 2. Represent each article as a vector of the 25 most important words in an article.
# 3. The distance between articles is calculated using the number of words that they have in common
# 4. Find the K-Nearest Neighbours and carry out a majority vote.

In [3]:
# This may get a bit hefty for Notepad, but we'll see.
# We'll control the counts from here:
article_count = 50
word_count    = 25
num_k         = 7

In [4]:
from urllib.request import urlopen
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from heapq import nlargest
from bs4 import BeautifulSoup
import re

In [5]:
ignore_list = set( stopwords.words('english')
                  + list(punctuation) 
                  + ['’',"'s","'it","'the","‘","'i","n't",'“','”','–','–','•','…','—'] 
                  + ['i','we','one','two','1','2','3'])

In [6]:
def extract_articles( link_prefix, link_list, extract_function, source_name, count ):
    processed = 0
    article_words = []
    for article_href in link_list:
        try:
            body_paragraphs = extract_function( link_prefix, article_href )
            if not body_paragraphs:
                raise ValueError('No body content found')
            article_blocks  = []
            for body in body_paragraphs:
                article_blocks.append( " ".join([ p.text.replace(u'\xa0', '') for p in body ]) )
            
            article_text = " ".join( article_blocks )
            word_freq = count_words( article_text, ignore_list )
            top_words = top_frequencies( word_count, word_freq, 0.9, 0.1 )
            
            article_words.append({ 'source':source_name, 'words': tuple(top_words), 'href': article_href })
            processed += 1
            
        except:
            #raise
            print('ERROR: ' + article_href)
            continue
        print( "{:5d}: {}".format(processed, article_href) )
        if processed >= count:
            break
            
    return article_words

In [7]:
def count_words( text, ignore_list ):
    word_freq = defaultdict(int)
    for word in word_tokenize(text):
        word_lc = word.lower();
        if word_lc not in ignore_list and word not in ignore_list:
            word_freq[ word_lc ] += 1
            #if word_freq[word_lc] > 2:
                #print('       word at {:3d}: {}'.format(word_freq[word_lc], word_lc) )
    return word_freq

In [8]:
def top_frequencies( n, freq_list, max_cut, min_cut ):
    freq_edit = dict( freq_list )
    max_freq  = float( max(freq_list.values()) )
    
    for word in freq_list.keys():
        word_freq = freq_list[word] / max_freq
        if word_freq >= max_cut or word_freq <= min_cut:
            del freq_edit[ word ]
    #print( freq_edit )
    return nlargest( n, freq_edit, key=freq_edit.get )

In [9]:
def process_links( article_links, count ):
    unique_links = set([ re.sub('#(.*)$', '', href) for href in article_links ])
    use_total    = min( count, unique_links.__len__() )
    print( "%d unique links found, trying to use using %d" % (unique_links.__len__(), use_total) )
    return list( unique_links )    

In [10]:
def soup_up( url ):
    home_content  = urlopen( url ).read().decode('utf8')
    home_soup     = BeautifulSoup( home_content, 'html.parser' )
    return home_soup

In [11]:
def crawlDailyMail( count, wordcount, ignore_list ):
    home_soup     = soup_up('http://www.dailymail.co.uk/home/index.html');
    article_links = [ a['href'] for a in home_soup.find_all('a', {'href': re.compile('/news/article-')}) ]
    link_list     = process_links( article_links, count )
    
    def extract_content( link_prefix, article_href ):
        article_soup = soup_up( link_prefix + article_href )
        article_body = article_soup.find_all('div', {'itemprop':'articleBody'})
        return [ a.find_all('p', {'class':'mol-para-with-font'}, recursive=False) for a in article_body ]
    
    return extract_articles(  'http://www.dailymail.co.uk', link_list, extract_content, 'daily mail', count )
        
print("Crawling the DM")
dm_vectors = crawlDailyMail( article_count, word_count, ignore_list )
#print( dm_vectors )

Crawling the DM
135 unique links found, trying to use using 50
    1: /news/article-5289257/Nigel-Farage-accused-secret-meetings-Assange.html
    2: /news/article-5291531/Retired-teacher-feeling-better-death.html
    3: /news/article-5290501/Fiancee-dead-Welsh-footballer-pay-60k-mortgage.html
    4: /news/article-5290297/Dog-travels-113-miles-train.html
    5: /news/article-5290909/Millions-drivers-struggling-pay-car-payments.html
    6: /news/article-5290091/Stormy-Daniels-claim-Donald-Trump-terrified-sharks.html
    7: /news/article-5290933/Drunk-British-Airways-pilot-hauled-Gatwick-flight.html
    8: /news/article-5289893/Fears-Carillion-2-6bn-pensions-black-hole.html
    9: /news/article-5290595/Care-home-chain-axes-death-fees-outcry.html
   10: /news/article-5290017/Kates-naughty-Uncle-Gary-Goldsmith-boozy-night-out.html
   11: /news/article-5288709/Christian-whipped-selling-alcohol-Islamic-law.html
   12: /news/article-5290567/ROBERT-HARDMAN-Peter-Mayle-Brits-fall-France.html
   

In [12]:
def crawlGuardian( count, wordcount, ignore_list ):
    home_soup     = soup_up('https://www.theguardian.com/uk-news');
    article_links = [ a['href'] for a in home_soup.find_all('a', { 'href': re.compile('/\d{4}/[a-z]{3}/\d{1,2}/') }) ]
    link_list     = process_links( article_links, count )
    
    link_list = list(map( lambda href: href.replace('https://www.theguardian.com', ''), link_list ))
    def extract_content( link_prefix, article_href ):
        article_soup = soup_up( link_prefix + article_href )
        article_body = article_soup.find_all('div', {'itemprop':'articleBody'})
        return [ a.find_all('p', recursive=False) for a in article_body ]
    
    return extract_articles( 'https://www.theguardian.com', link_list, extract_content, 'guardian', count )
    
print("Crawling the Guardian")
gd_vectors = crawlGuardian( article_count, word_count, ignore_list )
#print( gd_vectors )    

Crawling the Guardian
54 unique links found, trying to use using 50
ERROR: /uk-news/video/2018/jan/15/judgment-day-made-in-stoke-on-trent
    1: /theguardian/2018/jan/20/alexis-jay-ive-never-needed-therapy-i-did-get-angry-in-rotherham-though
    2: /uk-news/2018/jan/20/uk-drivers-warned-of-icy-conditions-as-wintry-weather-continues
    3: /business/2018/jan/11/does-london-mayor-sadiq-khans-brexit-report-stack-up
    4: /artanddesign/2018/jan/19/banksy-painting-saved-from-derelict-container-on-dungeness-beach
    5: /law/2018/jan/19/i-began-my-career-as-a-single-mum-female-qcs-changing-the-face-of-law
    6: /politics/2018/jan/13/nigel-farage-rattled-peers-brexit-struggle-now-beginning
    7: /politics/2018/jan/19/theresa-may-accused-of-timidity-in-tory-mps-outburst
    8: /world/2018/jan/20/womens-march-organisers-issue-rallying-call-to-britons
    9: /politics/2018/jan/19/mps-on-both-sides-back-tax-break-for-referendum-campaign-donors
   10: /politics/2018/jan/19/carl-sargeant-son-jac

In [13]:
def crawlIndependent( count, wordcount, ignorelist ):
    home_soup     = soup_up('http://www.independent.co.uk/');
    article_links = [ a['href'] for a in home_soup.find_all('a', { 'href': re.compile('^/news/(.*)\d{5,}.html') }) ]
    link_list     = process_links( article_links, count )
   
    def extract_content( link_prefix, article_href ):
        article_soup = soup_up( link_prefix + article_href )
        article_body = article_soup.find_all('div', {'itemprop':'articleBody'})
        return [ a.find_all('p', recursive=False) for a in article_body ]
    
    return extract_articles(  'http://www.independent.co.uk', link_list, extract_content, 'independent', count )

print('Crawling The Independent')
id_vectors = crawlIndependent( article_count, word_count, ignore_list )
#print( id_vectors )

Crawling The Independent
59 unique links found, trying to use using 50
    1: /news/world/asia/antelopes-saiga-population-why-extinct-mystery-solved-a8166966.html
    2: /news/health/nhs-intensive-care-pressure-highest-level-2010-swine-flu-pandemic-senior-doctors-a8169026.html
    3: /news/uk/crime/natalsha-gordon-suicide-pact-matthew-birkinshaw-left-die-alone-a8169386.html
    4: /news/uk/crime/oliver-mears-rape-case-surrey-police-cleared-lack-evidence-disclosure-oxford-university-student-a8168006.html
    5: /news/world/americas/us-politics/us-government-shutdown-latest-updates-donald-trump-blame-democrats-illegal-immigrants-military-a8169541.html
    6: /news/uk/politics/jean-claude-juncker-uk-eu-brexit-rejoin-referendum-president-article-49-a8163671.html
    7: /news/long_reads/hunter-s-thompson-death-suicide-kill-himself-how-die-gonzo-journalism-warren-hinckle-a8161841.html
    8: /news/world/americas/carl-higbie-trump-official-resigns-muslim-shithole-rant-us-president-a8167896.ht

In [14]:
def crawlExpress( count, wordcount, ignorelist ):
    home_soup     = soup_up('https://www.express.co.uk/news' );
    article_links = [ a['href'] for a in home_soup.find_all('a', { 'href': re.compile('^/news/[a-z]*/\d*/') }) ]
    link_list     = process_links( article_links, count )
    
    def extract_content( link_prefix, article_href ):
        article_soup = soup_up( link_prefix + article_href )
        article_body = article_soup.find_all('section', {'class':'text-description'})
        return [ a.find_all('p', recursive=False) for a in article_body ]
    
    return extract_articles(  'http://www.express.co.uk', link_list, extract_content, 'express', count )

print('Crawling The Express')
ex_vectors = crawlExpress( article_count, word_count, ignore_list )
#print( ex_vectors )

Crawling The Express
126 unique links found, trying to use using 50
    1: /news/world/907104/north-korea-news-japan-threat-kim-jong-un-abducted-Kazuhiro-Araki-world-war-3-ICC
    2: /news/world/907334/donald-trump-us-air-force-nuclear-bomber-B-21-raider-world-war-3
    3: /news/royal/907139/Buckingham-Palace-Queen-Elizabeth-George-VI-cigarettes-newspaper-history-Royals
    4: /news/politics/907087/labour-jeremy-corbyn-leftwing-labour-moderates-centrists-uk-politics-philip-collins-blair
    5: /news/uk/906980/Brexit-news-European-Union-EU-UK-referendum-Jean-Claude-Juncker-divorce-bill
    6: /news/politics/906940/US-government-shutdown-what-is-shut-down-means-for-USA
    7: /news/world/907124/North-Korea-news-latest-World-War-3-WW3-US-China-Donald-Trump-Kim-Jong-un-video
    8: /news/weird/907008/Pentagon-UFO-aliens-Nimitz-UFO-David-Fravor
    9: /news/world/907060/world-war-3-north-korea-nuclear-test-warning-mount-mantap-increased-tunnelling-activity
   10: /news/nature/907154/Orangut

In [15]:
class VectorSpace:
    
    def __init__( self, training_data, ignore_list ):
        self._training_data = training_data
        self._ignore_list   = ignore_list
    
    def _get_class_pc( self ):
        pass
    
    def test( self, text, wordcount, k ):
        neighbours = defaultdict(int)
        word_freq  = count_words( text, self._ignore_list )
        test_set   = set( word_freq )
        
        for article in self._training_data:
            neighbours[ (article['source'], article['href']) ] = len( set(article['words']).intersection(test_set) )
        
        k_nearest       = nlargest( k, neighbours, key=neighbours.get )
        classifications = defaultdict(int)
        for neighbour in k_nearest:
            classifications[ neighbour[0] ] += 1
        
        return [ (k, classifications[k]) for k in sorted(classifications, key=classifications.get, reverse=True) ]
        
training_data = dm_vectors + gd_vectors + id_vectors + ex_vectors
vs = VectorSpace( training_data, ignore_list )

In [16]:
# Let's test the BBC!
bbc_k = 7
count = 25 

print('Getting BBC links...')
bbc_soup  = soup_up('http://www.bbc.com/news')
bbc_links = [ a['href'] for a in bbc_soup.find_all('a', { 'href': re.compile('^/news/(.*)\d{6,}$') }) ]
link_list = process_links( bbc_links, count )

processed = 0;
for test_link in link_list:
    print('Testing ' + test_link )
    test_soup = soup_up('http://www.bbc.com' + test_link)
    print( test_soup.title.text )

    test_body    = test_soup.find('div', {'property':'articleBody'})
    if not test_body:
        count += 1
        print(' -- No test body found, continuing...\n')
        continue
    test_para    = test_body.find_all('p', recursive=False )
    if not test_para:
        print(' -- No paragraphs found, continuing...\n')
        continue
    
    test_text   = ' '.join([ p.text for p in test_para ])
    test_result = vs.test(test_text, word_count, bbc_k) 
    print(", ".join([ '{}% {}'.format(int(c[1]/bbc_k*100), c[0]) for c in test_result ]))
    print('')
    
    processed += 1
    if( processed >= count ):
        break
    


Getting BBC links...
41 unique links found, trying to use using 25
Testing /news/world-middle-east-42758532
Syria conflict: 15 refugees found frozen to death - BBC News
85% daily mail, 14% independent

Testing /news/world-us-canada-42758989
US shutdown: Senate reacts to failure to agree on new budget - BBC News
 -- No test body found, continuing...

Testing /news/world-europe-42749953
France: Emmanuel Macron’s marvellous manoeuvres this week - BBC News
42% daily mail, 28% guardian, 28% independent

Testing /news/world-asia-42729173
New Zealand debates access to dead sea life footage - BBC News
42% express, 28% independent, 28% guardian

Testing /news/world-us-canada-42738881
What is Trumpism? - BBC News
42% independent, 28% daily mail, 28% guardian

Testing /news/world-us-canada-42754608
What happens to federal workers in a government shutdown? - BBC News
 -- No test body found, continuing...

Testing /news/uk-england-sussex-42759124
'Drunk' pilot removed from BA plane at Gatwick Airpo